In [1]:
import pandas as pd
import warnings

from datetime import timedelta

warnings.simplefilter("ignore", pd.errors.SettingWithCopyWarning)

In [2]:
birds = pd.read_csv("../datasets/birds.csv")
birds

,traj_id,DateTime,lat,lon,Species
0,3552,08/01/2014 00:00:23,27.616280,90.226698,0
1,3552,08/01/2014 00:10:07,27.616251,90.226671,0
2,3552,08/01/2014 00:20:07,27.616210,90.226711,0
3,3552,08/01/2014 00:30:07,27.616251,90.226741,0
4,3552,08/01/2014 00:40:07,27.616348,90.226555,0
...,...,...,...,...,...
2650310,3920,05/10/2015 13:31:02,63.959604,43.918388,1
2650311,3920,05/10/2015 13:31:03,63.959761,43.918589,1
2650312,3920,05/10/2015 13:31:04,63.959915,43.918804,1
2650313,3920,05/10/2015 13:31:05,63.960074,43.919022,1


In [3]:
def shrink_dataset(dataset):
    final_dataset = []
    for id_ in dataset.traj_id.unique():
        # Get the smaller dataset.
        small = dataset.loc[dataset.traj_id == id_]
        small['DateTime'] = pd.to_datetime(small['DateTime'])
        days = small['DateTime'].dt.date.max() - small['DateTime'].dt.date.min()
        
        if days.days < 15:
            final_dataset.append(small)
        else:
            minimum = small['DateTime'].dt.date.min()
            maximum = minimum + timedelta(days=15)
            mask = (small['DateTime'].dt.date >= minimum) & (small['DateTime'].dt.date <= maximum)
            final_dataset.append(small.loc[mask])
    
    return pd.concat(final_dataset)

In [4]:
birds_small = shrink_dataset(birds)
birds_small

,traj_id,DateTime,lat,lon,Species
0,3552,2014-08-01 00:00:23,27.616280,90.226698,0
1,3552,2014-08-01 00:10:07,27.616251,90.226671,0
2,3552,2014-08-01 00:20:07,27.616210,90.226711,0
3,3552,2014-08-01 00:30:07,27.616251,90.226741,0
4,3552,2014-08-01 00:40:07,27.616348,90.226555,0
...,...,...,...,...,...
2636092,3920,2015-04-04 21:30:07,53.515127,27.749420,1
2636093,3920,2015-04-04 22:00:07,53.515127,27.749427,1
2636094,3920,2015-04-04 22:30:07,53.515094,27.749412,1
2636095,3920,2015-04-04 23:00:10,53.515087,27.749418,1


In [5]:
for id_ in birds_small.traj_id.unique():
    df = birds_small.loc[birds.traj_id == id_]
    df['DateTime'] = pd.to_datetime(df['DateTime']) # If your Date column is of the type object otherwise skip this
    date_range = df['DateTime'].dt.date.max() - df['DateTime'].dt.date.min()
    print(date_range.days)

13
15
15
15
15
15
15
10
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
11
15
15
15
8
15
15
15
15
15


In [ ]:
vultures = pd.read_csv("../datasets/vultures.csv")
vultures = vultures[['tag-local-identifier', 'timestamp', 'location-lat', 'location-long']]

vultures = vultures.rename(columns={
    "tag-local-identifier": 'traj_id', "timestamp": "DateTime", "location-lat": "lat", "location-long": "lon"
})
vultures['DateTime'] = pd.to_datetime(vultures.DateTime)
vultures.DateTime = vultures.DateTime.dt.strftime('%m/%d/%Y %H:%M:%S')
vultures['Species'] = 2
vultures = shrink_dataset(vultures)
vultures

In [ ]:
# goose = pd.read_csv("../datasets/goose.csv")
# goose = goose[['tag-local-identifier', 'timestamp', 'location-lat', 'location-long']]
# goose = goose.rename(columns={
#     "tag-local-identifier": 'traj_id', "timestamp": "DateTime", "location-lat": "lat", "location-long": "lon"
# })
# goose['DateTime'] = pd.to_datetime(goose.DateTime)
# goose.DateTime = goose.DateTime.dt.strftime('%m/%d/%Y %H:%M:%S')
# goose['Species'] = 1
# goose

In [ ]:
mixed = pd.concat([birds, vultures])
mixed

In [7]:
birds_small.to_csv("../datasets/birds_small.csv", index=False)